In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [2]:
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Splittting train dataset into train-val datasets

In [6]:
import splitfolders

splitfolders.ratio("./dino-dragon/train", output="./dino-dragon",
    seed=1337, ratio=(.8, .2), group_prefix=None, move=False)

Copying files: 1594 files [01:01, 26.02 files/s]


# Create Model

In [36]:
train_gen = ImageDataGenerator(rescale=1./255)

train_dataset = train_gen.flow_from_directory('./dino-dragon/train',
                              class_mode='binary',
                              target_size=(150,150),
                              batch_size=20,
                              shuffle=True)

val_gen = ImageDataGenerator(rescale=1./255)

val_dataset = val_gen.flow_from_directory('./dino-dragon/val',
                              class_mode='binary',
                              target_size=(150,150),
                              batch_size=20,
                              shuffle=True)

test_gen = ImageDataGenerator(rescale=1./255)

test_dataset = test_gen.flow_from_directory('./dino-dragon/test',
                              class_mode='binary',
                              target_size=(150,150),
                              batch_size=20,
                              shuffle=True)

Found 1274 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [29]:
def make_model():
    #load input datasets
    inputs = keras.Input(shape=(150,150,3))

    #add convolutional layer
    convolutional_layer = keras.layers.Conv2D(filters = 32,
                                              kernel_size = (3,3),
                                              activation='relu')(inputs)

    #add pooling layer
    pooling = keras.layers.MaxPool2D(pool_size=(2, 2))(convolutional_layer)
    #convert the results to a vector dataset
    vectors = keras.layers.Flatten()(pooling)
    #add a dense layer with 64 neurons
    dense = keras.layers.Dense(units = 64, activation = 'relu')(vectors)
    #create output dataset containing 1 neurons
    outputs = keras.layers.Dense(units = 1, activation = 'sigmoid')(dense)
    
    #create model
    model = keras.Model(inputs,outputs)
    #create optimizer
    optimizer = keras.optimizers.SGD(lr=0.002, momentum=0.8)
    #use BinaryCrossentripy for loss calculation
    loss = keras.losses.BinaryCrossentropy()

    model.compile(optimizer = optimizer,
             loss = loss,
             metrics = ['accuracy'])
    
    return model

## Q1: BinaryCrossentropy

## Q2: Total params: 11,215,873

In [27]:
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 150, 150, 3)]     0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 175232)            0         
_________________________________________________________________
dense_4 (Dense)              (None, 64)                11214912  
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 11,215,873
Trainable params: 11,215,873
Non-trainable params: 0
_______________________________________________

# Generate the model

In [38]:
checkpoint = keras.callbacks.ModelCheckpoint(
                    'model_dino_{epoch:02d}_{val_accuracy:.3f}.h5',
                     save_best_only= True,
                     monitor = 'val_accuracy',
                     mode = 'max')

model = make_model()

history = model.fit(train_dataset,
                    epochs=10 , 
                    validation_data= val_dataset,
                    callbacks = [checkpoint])

Epoch 1/10
64/64 [==============================] - 26s 404ms/step - loss: 0.7129 - accuracy: 0.5572 - val_loss: 0.6513 - val_accuracy: 0.5188
Epoch 2/10
64/64 [==============================] - 23s 361ms/step - loss: 0.6263 - accuracy: 0.6430 - val_loss: 0.5452 - val_accuracy: 0.7906
Epoch 3/10
64/64 [==============================] - 24s 379ms/step - loss: 0.5203 - accuracy: 0.7593 - val_loss: 0.4525 - val_accuracy: 0.8344
Epoch 4/10
64/64 [==============================] - 23s 367ms/step - loss: 0.4112 - accuracy: 0.8276 - val_loss: 0.4217 - val_accuracy: 0.8375
Epoch 5/10
64/64 [==============================] - 23s 357ms/step - loss: 0.3736 - accuracy: 0.8502 - val_loss: 0.3665 - val_accuracy: 0.8500
Epoch 6/10
64/64 [==============================] - 24s 376ms/step - loss: 0.3360 - accuracy: 0.8716 - val_loss: 0.3454 - val_accuracy: 0.8656
Epoch 7/10
64/64 [==============================] - 26s 408ms/step - loss: 0.3302 - accuracy: 0.8586 - val_loss: 0.3229 - val_accuracy: 0.8781

## Q3: median of training accuracy

In [51]:
import statistics
statistics.mean(history.history['accuracy'])

0.8102040827274323

## Q4: SD of training loss

In [55]:
import statistics
statistics.stdev(history.history['loss'])

0.14955097056199543

# Data augmentation

In [57]:
train_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest')

train_dataset = train_gen.flow_from_directory('./dino-dragon/train',
                              class_mode='binary',
                              target_size=(150,150),
                              batch_size=20,
                              shuffle=True)

val_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest')

val_dataset = val_gen.flow_from_directory('./dino-dragon/val',
                              class_mode='binary',
                              target_size=(150,150),
                              batch_size=20,
                              shuffle=True)

test_gen = ImageDataGenerator(rescale=1./255,
                               rotation_range=40,
                               width_shift_range=0.2,
                               height_shift_range=0.2,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode='nearest')

test_dataset = test_gen.flow_from_directory('./dino-dragon/test',
                              class_mode='binary',
                              target_size=(150,150),
                              batch_size=20,
                              shuffle=True)

Found 1274 images belonging to 2 classes.
Found 320 images belonging to 2 classes.
Found 394 images belonging to 2 classes.


In [60]:
loaded_model = keras.models.load_model('model_dino_07_0.878.h5')
new_history = loaded_model.fit(train_dataset,
                               validation_data=val_dataset,
                               epochs=10)

Epoch 1/10
64/64 [==============================] - 35s 544ms/step - loss: 0.4846 - accuracy: 0.7630 - val_loss: 0.4618 - val_accuracy: 0.7875
Epoch 2/10
64/64 [==============================] - 35s 542ms/step - loss: 0.4747 - accuracy: 0.7669 - val_loss: 0.4248 - val_accuracy: 0.8438
Epoch 3/10
64/64 [==============================] - 36s 568ms/step - loss: 0.4521 - accuracy: 0.7936 - val_loss: 0.5468 - val_accuracy: 0.7437
Epoch 4/10
64/64 [==============================] - 34s 540ms/step - loss: 0.4695 - accuracy: 0.7841 - val_loss: 0.4366 - val_accuracy: 0.7969
Epoch 5/10
64/64 [==============================] - 35s 543ms/step - loss: 0.4435 - accuracy: 0.7881 - val_loss: 0.3970 - val_accuracy: 0.8281
Epoch 6/10
64/64 [==============================] - 34s 528ms/step - loss: 0.4312 - accuracy: 0.8006 - val_loss: 0.4267 - val_accuracy: 0.7969
Epoch 7/10
64/64 [==============================] - 34s 539ms/step - loss: 0.4160 - accuracy: 0.8069 - val_loss: 0.3852 - val_accuracy: 0.8406

# Q5: mean of test loss

In [61]:
np.mean(new_history.history['val_loss'])

0.421614009141922

# Q6: avg of test accuracy for the last 5 epochs

In [64]:
np.mean(new_history.history['val_accuracy'][5:])

0.8306249856948853